# Bloque 1: Naturaleza de las series de tiempo

Melanie Oyarzún W.


**Resultado de aprendizaje esperado:**

Identificar datos de series temporales, sus particularidades y riesgos, en el contexto de posibles aplicaciones profesionales.

**Bibliografía recomendada:**

Stock & Watson, C.14 [link](https://www.dropbox.com/s/d09qcat340jy5q2/stock%20y%20watson%20-%20Introduccion%20a%20la%20Econometria.pdf?dl=0) ; Wooldridge, c.12 [link](https://www.dropbox.com/s/wrh0tuna27qp747/Wooldridge__Introductory_Econometrics_2nd_Ed_Solutions.pdf?dl=0), Gujarati, c.12  [link](https://www.dropbox.com/s/rrvth8vsa1c5gts/Gujarati-cap%2012.pdf?dl=0)

**Material de apoyo***

El taller cuenta con un repositorio  [https://github.com/melanieoyarzun/taller_IDS2021](https://github.com/melanieoyarzun/taller_IDS2021) en el cual está dispoinible todo el material.

**Slides**

Estan disponibles en el repositorio o directamente en el link 

---

In [ ]:
# Paquetes y settings

from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import watermark


#%matplotlib inline
%load_ext watermark

# setting de graficos

plt.figure(figsize=(5,3), dpi= 200, facecolor='w', edgecolor='k')


In [ ]:
%watermark -n -v -m -g -iv

# Tipos de datos

Los datos que analizamos con modelos se pueden categorizar en tres grandes tipos: 

- corte transversal 
- series de tiempo (o longitudinal)
- panel

## Corte transversal
Cuando se empieza a estudiar modelamiento de datos, la aplicación natural de estos es en lo que llamamos datos de corte transversal. En estos, la **unidad de observación** son individuos separados unos de otros. 

Usualmente, cada fila reprsenta un individuo (personas, familias, empresas) en un momento del tiempo específico (mismo año, mismo mes, etc). Es análogo a una *fotografía* de do grupo.


Un ejemplo de esto es la encuesta Casen [(http://observatorio.ministeriodesarrollosocial.gob.cl/encuesta-casen)](http://observatorio.ministeriodesarrollosocial.gob.cl/encuesta-casen) o Encuesta de Caracterización Socioeconómica Nacional.

Esta es una encuesta de caracter nacional realizada a hogares, en la cual se busca levantar información para conocer la situación de los hogares y de la población, especialmente de aquella en situación de pobreza y de aquellos grupos definidos como prioritarios y permite evaluar impacto de política social. Es representativa a nivel regional.

Es realizada por el Ministerio de Desarrollo Social desde el año 1990 con una periodicidad; bianual o trianual. Hasta ahora, las encuestas aplicadas corresponden a los años 1990, 1992, 1994, 1996, 1998, 2000, 2003, 2006, 2009, 2011, 2013, 2105 y 2017.

In [ ]:
df_casen2020= pd.read_stata("Data/casen_2020_ingresos.dta")
df_casen2020.head(10)

In [ ]:
df_casen2020.info()

In [ ]:

sns.scatterplot(data=df_casen2020, x="esc", y="yaut", alpha=0.5)

## Series de tiempo

En cambio, un tipo diferente de datos son las **Series Temporales** o tambien longitudinales.

En este tipo de información, se tiene diferentes momentos del tiempo (día, semana, mes, año, etc.) para una misma unidad de análisis (individuo, país, empresa, etc)

Ejemplos típicos de series de tiempo son: 
- Datos macroeconómicos (PIB. Inglación, empleo, etc.)
- Financieros (precios de acciones)
- Empresariales (Ventas, costos, etc...)


In [ ]:
#Ejemplo datos de acciones

#Usemos la api de Yahoo finance: pip install yahoo_fin y pip install requests_html

from yahoo_fin.stock_info import get_data

amazon_weekly= get_data("amzn", start_date="12/04/2009", end_date="12/04/2019", index_as_date = False, interval="1wk")
amazon_weekly


In [ ]:
sns.scatterplot(data=amazon_weekly, y="open", x="date")

#### Uso series de tiempo

La información temporal(de series de tiempo) permite responder cómo una variable (o más) responde a cambios a través del tiempo.
- ¿Cuál es el efecto causal dinámico de Xt sobre Yt? 
- ¿Cuál es la mejor predicción del valor de Y en el futuro?
  
**PERO su uso sin cuidado puede llevarnos a conclusiones MUY equiviocadas**


![Ejemplo idotamente peligroso](slide_figuras/spurius.png)


(Fuente: [https://www.tylervigen.com/spurious-correlations](https://www.tylervigen.com/spurious-correlations))

Trabajar con series de tiempo no es para nada sencillo: **necesitaremos técnicas nuevas.**

¿Por qué? 

Necesitaremos modelos sofisticados de errores y técnicas para estimarlos, por ciertas dieferencias en la naturaleza del **proceso generador de datos**

## Panel

El último tipo de datos, es lo que se llama un panel. Este es la combinacion de los dos tipos anteriores, donde para un conjunto de individuos tenemos varias observaciones en el tiempo.

Suele ser el tipo de datos más completo, pero tabien más dificil de conseguir. Además, enfrenta los problemas típicos de ambos tipos anteriores, dependiendo si es un panel corto o largo.

Una alternativa a etsos son los llamados pooled cross section.

Ela análisis de este tipo de datos escapa a los objetivos del taller, pero una introducción pueden revisarla en Stock y Watson Cap. 10.



# Corte transversal vs series de tiempo

## Muestras en corte transversal

En corte transversal solemos trabajar con **MUESTRAS**

![Corte transversal](slide_figuras/datos_st.png)

Estimamos modelos de la forma:
$$ y_{i}=\beta_{0}+\beta_{1}x_{1i}+\dots+\beta_{1}x_{ki}+u_{i} $$

Estos modelos representan una correlación marginal en las observaciones entre y y x's  (escalada por la varianza de x) y para que podamos interpretarlas causalmente tenemos varias condiciones o supuetsos que de sedeb cumplir.

Uno de estos, es el **supuesto de exogeneidad**: $$  E[u_{i}|X_{i}]=0 $$

Sin embargo, esta forma de ver este supuesto es una **simplificación** ya que, como estamos en una uestra aleatoria, no tenemos que verificar que los efectos cruzados tambien sean exogenos:

$$  E[u_{i}|X_{j}]=0 $$

* Esto se daba por cumplido, como consecuencia de que era una muestra aletaoria.
* Lo cual, generalente, ocurre en corte transversal por lo cual cada observación es i.i.d.




## La imposibilidad de la muestra en series de tiempo

En serie de tiempo, nuestro universo es un proceso estocástico:

![Corte transversal]("/Users/melanie/Dropbox/Asuntos UDD/2021- sem 2/taller_IDS2021/taller_IDS2021-1/slide_figuras/datos_st.png")

Estimamos modelos de la forma:

$$ y_{i}=\beta_{0}+\beta_{1}x_{1i}+\dots+\beta_{1}x_{ki}+u_{i} $$

* Estamos en presencia de un **proceso estocástico** 
* En cada momento se observa un posible resultado (o realización) del proceso estocásticos.

Esto tiene vaias implicancias:

* NO SON INDEPENDIENTES ya que por construcción, viene del mismo proceso.
* Por lo tanto, el supuesto de exogeneidad **$E[u_{i}|X_{i}]=0 $** NO ES SUFICIENTE
* Requerimos su versión más exigente: (Exogeneidad estricta)
$$E[u_{t}|X_{s}]=0 \qquad \forall s $$

* Este supuesto, generalmente NO SE CUMPLE.
* Si se cumpliese, seguiruíamos operando como siempre con modelos de regresión multiple estándar.

**¿Qué hacer entonces?**

Reconocer los datos como procesos estocásticos e incluir sus particularidades en la modelación. De eso se tratarán los dos siguientes bloques
- Bloque 2: Peculiaridades de los procesos estocasticos y su exploración en la data
- Bloque 3: Modelamiento



# Conceptos propios de las series de tiempo

Dado que una serie de tiempo tiene un orden específico, que en si mismo es importante. En base a este orden se suelen crear nuevos indicadores o variables. Revisemos los más comunes:

## Notación y transformaciones:

**Notación**

* Variables de series de tiempo se denominan con sub-índice t para indicar el perído en el tiempo:   $y_t$
* El total de periodos se suele referir como T.

### Rezagos 

* Un **rezago** es el valor de la variable en períodos anteriores:   
  * Primer rezago: $y_{t-1}$ es el valor 1 período anterior
  * Segundo rezago: $y_{t-2}$ es el valor 2 períodos atrás
  * j-ésimo rezago:  $y_{t-j}$ es el valor j períodos atrás

### Diferencias

Una diferencia corresponde al cambio en una variable entre dos periodos específicos y se usa la notación $\Delta$

* Primera diferencia: $$\Delta y_t = y_t- y_{t-1}$$

### Tasas de crecimiento

Si calculamos la primera diferencia el logaritmo natural, podemos obtener incorporar la tasa de crecimiento en una regresión:

* Primera diferencia en logs:
  $$ \Delta ln(y_{t})=ln(y_{t})-ln(y_{t-1}) $$

* Cambio porcentual de $y_t$  entre $t -1$ y $t\approx  100\times \Delta ln(y_{t})$

## Tipos de modelos

 ### Modelo estático 

 Se modela la relación contemporánea entre dos variables, i.e., relación en el mismo momento en el tiempo

$$y_{t}=\beta_{0}+\beta_{1}z_{t}+u_{t}$$

Ejemplos:
* Modelo Simple:
$$ inflación_{t}=\beta_{0}+\beta_{1}desempleo_{t}+u_{t}$$

* Modelo múltiple:
$$ homicidio_{t}=\beta_{0}+\beta_{1}condena_{t}+\beta_{2}desmepleo_{t}+\beta_{3}hombres_{t}+u_{t} $$

 ### Modelo dinámico

 Se incluyen efectos temporales, que se piensan que tienen que ver con **tendencias**, **inercia** o **estacionalidades**

 $$ inflación_{t}=\beta_{0}+\beta_{1}desempleo_{t}+\beta_{2} inflación_{t-1} +u_{t}$$

# Pandas y series de tiempo

* Pandas fue desarrollado originalmente para trabajar con datos financieros.
* Como las series temporales son un tipo de datos comunes encontrados en aplicaiones de finanzas, naturalmente pandas tiene muy buen soporte para la mayoria de operaciones comunes.


* `pd.to_datetime()` : conviere una serie o valor en un timestamp
  * Este formato permite un mejor manejo de las series.

In [ ]:
#pandas remote data access support for calls to the World Bank Indicators API
from pandas_datareader import data, wb #instalar conda install pandas-datareader  o  pip installpandas-datareader

In [ ]:
#Revisemos que indicadores hay disponibles. En este caso revisare de PIB (GDP en ingés), pero se pueden explorar muchas más opciones.

wb.search('gdp.*capita.*const')


In [ ]:
# Obtengamos la lista de paises disponibles
countries=wb.get_countries()

#Preview primeras filas lista de paises
countries[:5]

In [ ]:
#sabemos que queremos Chile, asi que busquemos su info

countries[ countries['name'] == 'Chile' ]


In [ ]:
# Descarguemos la data desde la API del banco muncial a un dataframe

df_GPDpc_Chile = wb.download(
                    #Use the indicator attribute to identify which indicator or indicators to download
                    indicator='NY.GDP.PCAP.KD',
                    #Use the country attribute to identify the countries you want data for
                    country=['CL'],
                    #Identify the first year for which you want the data, as an integer or a string
                    start='1980',
                    #Identify the last year for which you want the data, as an integer or a string
                    end=2020,
                )

#Veamos el data frame
df_GPDpc_Chile.head(10)

In [ ]:
wb.download( indicator=['NY.GDP.PCAP.PP.KD','NY.GDP.PCAP.KD'], country=['CL'], start=2008, end=2010 )

In [ ]:
sns.scatterplot(data=df_GPDpc_Chile, x="year", y="NY.GDP.PCAP.KD")

In [ ]:
ax = df_GPDpc_Chile.plot(legend=True)

# Ejemplos

## 1. PIB de EEUU

Empecemos con un ejemplo muy clásico de series de tiempo, con datos del PIB de Estados Unidos

In [ ]:
GDP = pd.read_csv('data/GDP.csv', parse_dates=['DATE'])
GDP.info()

In [ ]:
GDP.set_index('DATE', inplace=True)
GDP.info()

In [ ]:
GDP.head()

In [ ]:
ax = GDP['2009':].plot(legend=False)
ax.set_ylabel(r'GDP ($\$B$)')

Observamos claramente una **tendencia** al alza.

## 2. Mortalidad por influenza

Vemos un segundo ejemplo 

In [ ]:
ILI = pd.read_csv('data/CDC.csv')
ILI.head()

In [ ]:
ILI['date'] = ILI['Year']+ILI['Week']/52.
ILI.head()

In [ ]:
ILI.plot(x='date', y=['Percent of Deaths Due to Pneumonia and Influenza', 'Expected', 'Threshold'])
ax = plt.gca()
ax.legend(['Mortality', 'Expected', 'Threshold'])
ax.set_xlabel('Date')
ax.set_ylabel('% Mortality')

El comportamiento es muy diferente, podemos ver una **estacionalidad**.

Es decir hay un cambio periodico asociado al tiempo de la variable.

# Transformaciones

En base a estos conceptos, entonces, es muy comun realizar transformaciones a las series de tiempo ya sea para obtener rezagos, diferencias, tasas de crecimiento, etc.
Ilustraremos varios de estos, con el dataset del DOw-Jonses indsutrial Average.


In [ ]:
DJIA = pd.read_csv('data/DJIA.csv', parse_dates=['DATE'], na_values='.').dropna()
DJIA.plot(x='DATE', legend=False)
ax = plt.gca()
ax.set_ylabel('DJIA')
ax.set_xlabel('Date')



## Diferencias

Una manera de remover tendencias de un dataset es diferenciandolo. Como nuestros datasets son discretos, usaremos diferencias finitas.

In [ ]:
def differentiate(values, d=1):
    # First value is required so that we can recover the original values with np.cumsum
    x = np.concatenate([[values[0]], values[1:]-values[:-1]])

    if d == 1:
        return x
    else:    
        return difference(x, d - 1)

In [ ]:
values = DJIA['DJIA'].values
differences = differentiate(values)

Como podemos ver, el grafico se ve mucho más estacionario.

In [ ]:
plt.plot(DJIA['DATE'].iloc[1:], differences[1:])
plt.xlabel('Date')
plt.ylabel('Differences')

Para recuperar la data original, basta con integrar los puntos diferenciados. 

In [ ]:
def integrate(values, d=1):
    x = np.cumsum(values)
    
    if d == 1:
        return x
    else:
        return integrate(x, d-1)

In [ ]:
rebuilt = integrate(differences)

Y un chequeo rapido para ver que son los mismos valores

In [ ]:
np.mean(rebuilt-values)

## Windowing
We also often want to calculate running values of some quantity. This requires the use of windowing functions that return the proper element at each step. 
def rolling(x, order):
    npoints = x.shape[0]
    running = []
    
    for i in range(npoints-order+1):
        running.append(x[i:i+order])
        
    return np.array(running)
And a simple example
values = np.arange(11)
values
rolling(values, 6)
Since we return a numpy array with all the individual windows, this also provides us with a simple way to take running averages by chaining methods
rolling(values, 2)
rolling(values, 2).mean(axis=1)
Or the running maximum, etc.
rolling(values, 2).max(axis=1)

## Exponential Smoothing
Another form of smoothing a noise time series is called exponential smoothing. This is equivalent to an exponentially weighted running average where past values get exponentially reduced.
def ES(values, alpha= 0.05):
    N = len(values)
    S = [values[0]*alpha]
    
    for i in range(1, N):
        S.append(alpha*values[i]+(1-alpha)*S[-1])
        
    return np.array(S)
As we can see with a few quick examples, the smaller the value of alpha the smoother (less noisy) the result
smooth = []
smooth.append(ES(differences[1:], 0.01))
smooth.append(ES(differences[1:], 0.1))
smooth.append(ES(differences[1:], 0.5))
plt.plot(DJIA['DATE'].iloc[1:100], differences[1:100], label='Differences')
plt.plot(DJIA['DATE'].iloc[1:100], smooth[2][:99], label=r'$\alpha=0.5$')
plt.plot(DJIA['DATE'].iloc[1:100], smooth[1][:99], label=r'$\alpha=0.1$')
plt.plot(DJIA['DATE'].iloc[1:100], smooth[0][:99], label=r'$\alpha=0.01$')
plt.xlabel('Date')
plt.ylabel('Differences')
plt.legend()

## Missing Data

Desafortunadamente, los datos no siempre están limpios o completos, lo que nos obliga a lidiar datos faltantes. Aquí ilustramos varios enfoques para introducir valores perdidos. Comenzamos generando un conjunto de datos con valores perdidos.


In [ ]:
x = np.linspace(-np.pi, np.pi, 100)
y = np.cos(x)
y_missing = y.copy()
y_missing[40:55] = np.nan
This is simply a cosine function with a few missing values at the peak.
plt.plot(x, y, '*')
plt.plot(x, y_missing)


Quizás la estrategia más común es simplemente mantener el último valor 'bueno' conocido y usarlo para completar los puntos de datos faltantes. Este enfoque no puede lidiar con los valores faltantes al comienzo del conjunto de datos. 


In [ ]:
def ffill(y):
    y0 = y.copy()
    N = len(y0)
    
    current = None
    for i in range(1, N):
        if np.isnan(y0[i]):
            y0[i] = current
        else:
            current = y0[i]
    
    return y0


Naturalmente, el enfoque opuesto también es común cuando usamos el siguiente valor bueno. De esta manera podemos manejar fácilmente los valores perdidos iniciales, pero no podemos hacer nada con los valores perdidos al final de la serie de tiempo.



In [ ]:

def bfill(y):
    y0 = y.copy()
    N = len(y0)
    
    current = None
    for i in range(N-1, 0, -1):
        if np.isnan(y0[i]):
            y0[i] = current
        else:
            current = y0[i]
    
    return y0


Back-fill y Forward-fill son enfoques simples pero poderosos para lidiar con los datos faltantes. Sin embargo, a menudo queremos tener más cuidado con el valor que atribuimos. Un enfoque común es interpolar entre el valor anterior y el siguiente y conectarlos con una línea recta.



In [ ]:
def interpolate(y):
    y0 = y.copy()
    N = len(y0)
    
    pos = 0
    while pos < N:
        if np.isnan(y0[pos]):
            count = 0
            
            while np.isnan(y0[pos+count]):
                count += 1
            
            current = y0[pos-1]
            future = y0[pos+count]
            slope = (future-current)/count
            
            y0[pos:pos+count] = current + np.arange(1, count+1)*slope
            
            pos += count
        else:
            pos += 1
            
    return y0


La imputación de datos (el cálculo de los valores perdidos esperados) es un gran subcampo de estadísticas con una amplia gama de técnicas y enfoques. 

In [ ]:


y_bfill = bfill(y_missing)
y_ffill = ffill(y_missing)
y_inter = interpolate(y_missing)
And a quick plot to visualize the differences
plt.plot(x, y_bfill, label='back fill')
plt.plot(x, y_ffill, label='forward fill')
plt.plot(x, y_inter, label='interpolate')
plt.plot(x, y_missing, label='Data')
plt.legend()

## Resampling

En muchos casos, también necesitamos cambiar la frecuencia a la que estamos operando. Por ejemplo, nuestro conjunto de datos DJIA tiene valores al final del día, pero podríamos estar interesados en puntos de datos semanales o mensuales. El remuestreo es una serie de técnicas diseñadas para lidiar con esta situación y es similar en espíritu a las técnicas de ventanas que vimos anteriormente. La principal diferencia es que en lugar de simplemente mover la ventana en un paso fijo, cada ventana corresponde a nuestro período de interés.


In [ ]:
mapping = DJIA['DATE'].dt.year
values = DJIA['DJIA'].values



En el caso más simple, simplemente calculamos cuál es la ventana correcta para cada punto de datos y la agregamos en consecuencia.

In [ ]:

def groupBy(values, mapping, func = None):
    agg = {}
    pos = {}
    
    for i in range(values.shape[0]):
        key = mapping.iloc[i]
        
        if key not in agg:
            agg[key] = []
        
        pos[key] = i
        
        if not np.isnan(values[i]):
            agg[key].append(values[i])
        
    order = sorted(agg.keys())
    
    if func is not None:
        for key in agg:
            agg[key] = func(np.array(agg[key]).astype('float'))
            
    return agg, pos


Naturalmente, esta función groupBy es útil no solo para remuestrear sino también para una amplia gama de análisis estadísticos. Además de un mapeo, también debemos especificar qué función de agregación queremos usar. ¿Nos interesa el valor medio? ¿el maximo? ¿Desviación Estándar?

In [ ]:
agg, pos = groupBy(values, mapping, np.mean)

Aca simplemente calculamos la media por año

In [ ]:
agg

Como nuestra función groupBy también devuelve las posiciones de índice de la última vez que se vio cada bin, podemos comparar fácilmente los datos originales con los muestreados nuevamente.

In [ ]:

aggregated = []

for key in pos:
    aggregated.append([pos[key], agg[key]])

aggregated = np.array(aggregated)
aggregated
plt.plot(DJIA['DATE'], DJIA['DJIA'])
ax = plt.gca()
ax.plot(DJIA.set_index('DATE').index[aggregated.T[0].astype('int')], aggregated.T[1])
ax.plot(DJIA.set_index('DATE').index[aggregated.T[0].astype('int')], aggregated.T[1], 'ro', markersize=10,)
ax.set_ylabel('DJIA')
ax.set_xlabel('Date')

## Jackknife estimators

FFinalmente, en muchos casos queremos estimar cantidades estadísticas de series de tiempo. Un ejemplo obvio podría ser estimar la media móvil de una serie sin tendencia para verificar si de hecho está lo suficientemente cerca de cero para ser considerada estacionaria.

El estimador JackKnife nos permite obtener no solo el valor esperado en cuestión, sino también una medida de su varianza. Esto se logra mediante el uso de un enfoque de omisión para calcular N estimaciones de nuestra métrica. A partir de esta población de estimación, podemos obtener los promedios como la mejor estimación posible y la desviación estándar como una medida de las barras de error involucradas.


In [ ]:
def jackknife(x, func, variance = False):
    N = len(x)
    pos = np.arange(N)
    values = [func(x[pos != i]) for i in pos]
    jack = np.sum(values)/N
    
    if variance:
        values = [np.power(func(x[pos != i]) - jack, 2.0) for i in pos]
        var = (N-1)/N * np.sum(values)
        return jack, var
    else:
        return jack

In [ ]:
x = np.random.normal(0, 2, 100)
print(x.std())
jackknife(x, np.std, True)


Con jackknife obtenemos no solo una estimación del valor sino también una medida del error

## Bootstrapping

Otra técnica común para estimar propiedades estadísticas se conoce como bootstrapping y está estrechamente relacionada con Jackknife. En este enfoque, simplemente tomamos muestras (con reemplazo) de la población original para obtener una medida de cuánta variabilidad se puede esperar.


In [ ]:
def bootstrapping(x, n_samples, func=np.mean):
    y = x.copy()
    N = len(y)
    population = []
    
    for i in range(n_samples):
        population.append(func(np.random.choice(y, N, replace=True)))
        
    return np.array(population)


Podemos generar fácilmente un histograma de las muestras bostrappeadas

In [ ]:
def histogram(values, n_bins=100):
    xmax = values.max()
    xmin = values.min()
    delta  = (xmax-xmin)/n_bins
    
    counts = np.zeros(n_bins+1, dtype='int')
    
    for value in values:
        val_bin = np.around((value-xmin)/delta).astype('int')
        counts[val_bin] += 1.0
    
    bins = xmin+delta*np.arange(n_bins+1)
    
    return bins, counts/values.shape[0]


In [ ]:
  
x = np.random.normal(0, 2, size=100)


In [ ]:

boot = bootstrapping(x, 1000)


In [ ]:

x.mean()


In [ ]:
x.std()


In [ ]:
bins, counts = histogram(boot)


In [ ]:
plt.plot(bins, counts)
plt.vlines(x=boot.mean(), ymin=0, ymax=counts.max(), label='mean', color='g')
plt.vlines(x=boot.mean()+boot.std(), ymin=0, ymax=counts.max(), label='std', linestyles='--', color='r')
plt.vlines(x=boot.mean()-boot.std(), ymin=0, ymax=counts.max(), label='std', linestyles='--', color='r')


# Actividad Bloque 1:

1. Siga este ejemplo practico de importar datos desde la API del Banco Mundial y preparar la base para su análisis de series de tiempo.
2. Importe usted la serie de GDP total Y Percapita para otro país serie desde la API del Banco mundial, muestre sus principales características y realice un grafico. 
3. Obtenga las primeras diferencias y compare, ¿pareciera haber tendencias?

In [ ]:
#pandas remote data access support for calls to the World Bank Indicators API
from pandas_datareader import data, wb #instalar conda install pandas-datareader  o  pip installpandas-datareader

#Revisemos que indicadores hay disponibles. En este caso revisare de PIB (GDP en ingés), pero se pueden explorar muchas más opciones.

wb.search('gdp.*capita.*const')


In [ ]:
# Obtengamos la lista de paises disponibles
countries=wb.get_countries()

#Preview primeras filas lista de paises
countries[:5]

In [ ]:
#sabemos que queremos Chile, asi que busquemos su info

countries[ countries['name'] == 'Chile' ]

In [ ]:
# Descarguemos la data desde la API del banco muncial a un dataframe

df_GPDpc_Chile = wb.download(
                    #Use the indicator attribute to identify which indicator or indicators to download
                    indicator='NY.GDP.PCAP.KD',
                    #Use the country attribute to identify the countries you want data for
                    country=['CL'],
                    #Identify the first year for which you want the data, as an integer or a string
                    start='1980',
                    #Identify the last year for which you want the data, as an integer or a string
                    end=2020
                )

#Veamos el data frame
df_GPDpc_Chile.head(10)

In [ ]:
wb.download( indicator=['NY.GDP.PCAP.PP.KD','NY.GDP.PCAP.KD'], country=['CL'], start=2008, end=2010 )

In [ ]:
sns.scatterplot(data=df_GPDpc_Chile, x="year", y="NY.GDP.PCAP.KD")

Transformemos la columna de tiempo a un indice en el dataframe



In [ ]:
df_GPDpc_Chile.index=pd.to_datetime(df_GPDpc_Chile['year'], format="%Y" )